# Create album

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.CNN
from itertools import combinations 
from joblib import parallel_backend

In [ ]:
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [ ]:
# Choose your responses, size, for the image and number of features we are predicting
pixels = 40
features = 62

In [ ]:
#Decide which dataset you want to predict
resp_n = 'immune'
response = [col for col in DF if col.startswith(resp_n)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [ ]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [ ]:
predictors = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']
predictors.remove(resp_n)

In [ ]:
albums = Parallel(n_jobs=6, prefer = "threads")(delayed(butterfly.album.create_album)
                                                (DF, predictors[al], pixels) 
                                                for al in range(6))

In [ ]:
yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values

In [ ]:
bars_p = []
bars_o = []

for pred in range(6):

    prediction, observed = zip(*Parallel(n_jobs=len(exclude), prefer="threads")(delayed(butterfly.CNN.model)
                                                               (response_df, exclude, cv, albums[pred], pixels,features) 
                                                               for cv in range(len(exclude))))
                
    bars_p.append(pd.concat(prediction))
    bars_o.append(pd.concat(observed))

In [ ]:
results = []
for res in range(6):
    correl = bars_p[res].corrwith(bars_o[res], axis = 0) 
    correl[correl < 0] = 0
    results.append(np.mean(correl))

In [ ]:
results